# Training

### Library import

In [ ]:
from super_gradients import Trainer
from super_gradients.training import Trainer, training_hyperparams
from super_gradients.training import models
from super_gradients.common.object_names import Models

In [ ]:
import os
import pprint
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.transforms import Compose
from matplotlib import pyplot as plt

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

### Paths organization

In [ ]:
# Dataset
pwd_notebook = os.path.abspath('') # path notebook
root_path = os.path.dirname(os.path.dirname(pwd_notebook)) #path root project

data_dir = os.path.join(root_path, 'data') #path data

# train path
train_dir = os.path.join(data_dir, 'train')
print(train_dir)
# test path
test_dir = os.path.join(data_dir, 'test')
print(test_dir)
#valid path
valid_dir = os.path.join(data_dir, 'valid')
print(valid_dir)

In [ ]:
# Checkpoints
experiment_dir = os.path.join(root_path, 'checkpoints', 'class_mam')
CHECKPOINT_DIR = os.path.join(experiment_dir, "class_ckpt")
model_name = 'resnet18'
trainer = Trainer(experiment_name=experiment_dir, ckpt_root_dir=CHECKPOINT_DIR)
trainer

### Transformation

In [ ]:
# Write transform for image
data_transform = Compose([
    # Resize the images to 64x64
    # transforms.Resize(size=(64, 64)),
    # Flip the images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5), # p = probability of flip, 0.5 = 50% chance
    # Turn the image into a torch.Tensor
    transforms.ToTensor() # this also converts all pixel values from 0 to 255 to be between 0.0 and 1.0 
])

### Assigning train, test and valid

In [ ]:
train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=data_transform, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)

valid_data = datasets.ImageFolder(root=valid_dir, 
                                 transform=data_transform,
                                 target_transform=None)

test_data = datasets.ImageFolder(root=test_dir, 
                                 transform=data_transform,
                                 target_transform=None)

print(f"Train data:\n{train_data}\nValid data:\n{valid_data}\nTest data:\n{test_data}")

In [ ]:
# Get class names as a list
class_names = train_data.classes
print(class_names)
# Can also get class names as a dict
class_dict = train_data.class_to_idx
print(class_dict)
# Check the lengths
len(train_data), len(valid_data)

In [ ]:
img, label = train_data[5][0], train_data[5][1]
print(f"Image tensor:\n{img}")
print(f"Image shape: {img.shape}")
print(f"Image datatype: {img.dtype}")
print(f"Image label: {label}")
print(f"Label datatype: {type(label)}")

In [ ]:
# Rearrange the order of dimensions
img_permute = img.permute(1, 2, 0)

# Print out different shapes (before and after permute)
print(f"Original shape: {img.shape} -> [color_channels, height, width]")
print(f"Image permute shape: {img_permute.shape} -> [height, width, color_channels]")

# Plot the image
plt.figure(figsize=(10, 7))
plt.imshow(img.permute(1, 2, 0))
plt.axis("off")
plt.title(class_names[label], fontsize=14)

### Training settings

In [ ]:
train_dataloader = DataLoader(dataset=train_data, 
                              batch_size=16, # how many samples per batch?
                              num_workers=10, # how many subprocesses to use for data loading? (higher = more)
                              shuffle=True) # shuffle the data?

valid_dataloader = DataLoader(dataset=valid_data, 
                             batch_size=16, 
                             num_workers=10, 
                             shuffle=True) # don't usually need to shuffle testing data

test_dataloader = DataLoader(dataset=test_data, 
                             batch_size=16, 
                             num_workers=20, 
                             shuffle=False) # don't usually need to shuffle testing data

train_dataloader, valid_dataloader, test_dataloader

In [ ]:
img, label = next(iter(train_dataloader))

# Batch size will now be 1, try changing the batch_size parameter above and see what happens
print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label.shape}")
print(f"Train data:\n{train_data}\nValid data:\n{valid_data}\nTest data:\n{test_data}")

### Model

In [ ]:
# "num_classes = " change to your number of classes
model = models.get(model_name=Models.RESNET18, num_classes=6, pretrained_weights="imagenet")
print(model.train)

In [ ]:
# you can see more recipes in super_gradients/recipes
training_params =  training_hyperparams.get("training_hyperparams/imagenet_resnet50_train_params")

pprint.pprint("Training parameters")
pprint.pprint(training_params)

In [ ]:
# Add the epochs you want
training_params["max_epochs"] = 5
training_params["sg_logger_params"]["launch_tensorboard"] = True

### Training visualization

In [ ]:
log_dir = os.path.join(root_path, 'logs', 'training_logs')

%load_ext tensorboard
%tensorboard --logdir log_dir

### Train

In [ ]:
trainer.train(model=model,
              training_params=training_params,
              train_loader=train_dataloader,
              valid_loader=valid_dataloader)
